## Depencies

In [ ]:
%pip install numpy
%pip install matplotlib

In [ ]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from IPython.display import HTML

## Leg Mechanism

In [ ]:
class LegBase:
  def __init__(self, angle0_init=2*np.pi/3, angle5_init=np.pi/3):
    self.L0 = 8.16
    self.L1 = 8.0
    self.L2 = 10.0
    self.L3 = 8.0
    self.L4 = 13.0
    self.L5 = 8.0
    self.angle0 = angle0_init
    self.angle5 = angle5_init

## Kinematics Analysis in $R^2$

In [ ]:
class LegKinematics2D(LegBase):
  def __init__(self):
    super().__init__()

### Forward Kinematics

In [86]:
def calc_ang2pnt(self, angle0, angle5):
  '''
  Forward kinematics
  Given the angles of the motors, calculate the state of the leg, including the position of the points and the angles of the joints
  Return the points and angles of the joints
  '''
  p0 = np.array([0, 0])
  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p5 = np.array([self.L0, 0])
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5
  L14 = np.linalg.norm(p4-p1)

  if(L14 > self.L4 + self.L2):
    print("Unreachable")
    return None
  elif (L14 == self.L4 + self.L2):
    print("Singularity")

  angle_214 = acos((self.L2**2 + L14**2 - self.L4**2) / (2 * self.L2 * L14))
  rotation_matrix1 = np.array([[np.cos(angle_214), -np.sin(angle_214)],
                              [np.sin(angle_214), np.cos(angle_214)]])
  rotation_matrix2 = np.array([[np.cos(-angle_214), -np.sin(-angle_214)],
                              [np.sin(-angle_214), np.cos(-angle_214)]])
  
  unit_vector_14 = (p4 - p1) / L14
  p2_1 = p1 + self.L2 * rotation_matrix1 @ unit_vector_14
  p2_2 = p1 + self.L2 * rotation_matrix2 @ unit_vector_14
  p2 = p2_1 if p2_1[1] < p2_2[1] else p2_2

  # Translate p2 to p3
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  # Store the states
  points = np.array([p0, p1, p2, p3, p4, p5])

  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5
  ])

  return points, angles

LegKinematics2D.calc_ang2pnt = calc_ang2pnt

Visualizing the leg

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

# Initialize the points and lines
legkinematics = LegKinematics2D()
points = np.zeros((6, 2))  # 6 points in 2D
scat = ax.scatter(points[:, 0], points[:, 1], color='r')
line1, = ax.plot([], [], color='b') # Line p0 -> p1 -> p2 -> p4 -> p5
line2, = ax.plot([], [], color='b') # Line p2 -> p3

# Update function for animation
angle0 = np.linspace(np.pi, np.pi/2, 200)
angle5 = np.linspace(np.pi/2, 0, 200)

def update(frame):
  ang0 = angle0[frame]
  ang5 = angle5[frame]
  points[:6] = legkinematics.calc_ang2pnt(ang0, ang5)[0]

  # Update scatter plot
  scat.set_offsets(points)

  # Update lines
  line1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
  line2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])

  return scat, line1, line2

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_xlim([-15, 20])
ax.set_ylim([-30, 5])
ax.set_aspect('equal')

ani = FuncAnimation(fig, update, frames=200, interval=20, blit=True)
HTML(ani.to_jshtml())

### Inverse Kinematics

In [88]:
def calc_pnt2ang(self, x, y):
  '''
  Calculate the inverse kinematics using the geometric method.
  Given the position of the foot, calculate the angles of the motors,
  Return the angles of the motors, which are p0 and p5.
  '''

  # Calculate the angle of the motor 0
  L03 = np.linalg.norm(np.array([x, y]))
  angle_305 = atan2(-y, x)
  if angle_305 < 0:
    angle_305 += np.pi
  angle_301 = acos((self.L1**2 + L03**2 - (self.L2 + self.L3)**2) / (2 * self.L1 * L03))
  angle0 = angle_305 + angle_301

  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p2 = (np.array([x, y]) * self.L2 + p1 * self.L3) / (self.L2 + self.L3)
  p5 = np.array([self.L0, 0])

  # Calculate the angle of the motor 5
  L25 = np.linalg.norm(p2 - p5)
  angle_250 = atan2(-p2[1], self.L0 - p2[0])
  if angle_250 < 0:
    angle_250 += np.pi
  angle_254 = acos((L25**2 + self.L5**2 - self.L4**2) / (2 * L25 * self.L5))
  angle5 = np.pi - (angle_250 + angle_254)

  # Store the states
  p0 = np.array([0, 0])
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  points = np.array([p0, p1, p2, p3, p4, p5])

  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5
  ])

  return angles, points

LegKinematics2D.calc_pnt2ang = calc_pnt2ang


Check if inverse kinematics fits the forward kinematics

In [ ]:
legkinematics = LegKinematics2D()
angle0, angle5 = 2*np.pi/3, np.pi/3
print("Forward kinematics")
print("Given Angles: ", angle0, angle5)

points, angles = legkinematics.calc_ang2pnt(angle0, angle5)
print("Points: ", points)

print("\nInverse kinematics")
angles, points = legkinematics.calc_pnt2ang(points[3][0], points[3][1])
print("Given Point: ", points[3])
print("Angle0: ", angles[0], ", Angle5: ", angles[5])

### Differential Forward Kinematics

In [ ]:
def calc_omg2vel(self, omega0, omega5, angles):
  '''
  Calculate the velocity of the foot given the angular velocity of the motors
  '''
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])
  omega1 = omega0 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  vx2 = -self.L1 * omega0 * sin(angles[0]) - self.L2 * omega1 * sin(angles[1])
  vy2 = -self.L1 * omega0 * cos(angles[0]) - self.L2 * omega1 * cos(angles[1])

  vx3 = vx2 - self.L3 * omega1 * sin(angles[1])
  vy3 = vy2 - self.L3 * omega1 * cos(angles[1])
  
  return vx3, vy3

LegKinematics2D.calc_omg2vel = calc_omg2vel


Check if the result of the differential forward kinematics fit that of the forward kinematics. If it does, you will see only one vector in the left plot and two curves in the right plot, because the calculated values are the same as the actual values.

In [ ]:
class Verifier2D:
  def __init__(self):
    self.fig, (self.ax_mech, self.ax_vel) = plt.subplots(1, 2, figsize=(15, 5))

    # Set up axes
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_aspect('equal')
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')
    # self.ax_mech.legend()
    
    self.ax_vel.set_xlim(0, 2)  # Match your total_time
    self.ax_vel.set_ylim(-20, 20)
    self.ax_vel.grid(True)
    self.ax_vel.set_title('P3 Velocity Components')
    self.ax_vel.set_xlabel('Time (s)')
    self.ax_vel.set_ylabel('Velocity')
    # self.ax_vel.legend()

    # Data storage
    self.times = []
    self.actual_velocities = []
    self.calculated_velocities = []
    self.points_history = []

    # Initialize empty lines for the mechanism
    self.line1, = self.ax_mech.plot([], [], 'blue', lw=2, label='Chain 1')
    self.line2, = self.ax_mech.plot([], [], 'blue', lw=2, label='Chain 2')
    self.joints, = self.ax_mech.plot([], [], 'ro', label='Joints')
    self.calc_vector = self.ax_mech.quiver([], [], [], [], color='r', scale=1, units="xy", label='V Actual')
    self.actual_vector = self.ax_mech.quiver([], [], [], [], color='g', scale=1, units="xy", label='V Calc')

    # Initialize velocity plot lines
    self.vx_actual, = self.ax_vel.plot([], [], label='Vx')
    self.vy_actual, = self.ax_vel.plot([], [], label='Vy')
    self.vx_calc, = self.ax_vel.plot([], [], label='Vx Calc')
    self.vy_calc, = self.ax_vel.plot([], [], label='Vy Calc')
    
    self.ax_mech.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
    self.ax_vel.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

  def update(self, frame):
    points = self.points_history[frame]
    
    # Update chain
    self.line1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
    self.line2.set_data(points[[0, 5, 4, 2], 0], points[[0, 5, 4, 2], 1])
    self.joints.set_data(points[:, 0], points[:, 1])

    # Update velocity vector at p3
    if frame > 0:
      actual_vel = self.actual_velocities[frame]
      calc_vel = self.calculated_velocities[frame]

      # Plot velocity vector
      self.calc_vector.set_offsets(points[3])
      self.calc_vector.set_UVC(calc_vel[0], calc_vel[1])
      self.actual_vector.set_offsets(points[3])
      self.actual_vector.set_UVC(actual_vel[0], actual_vel[1])
        
    # Plot velocity components over time
    time = self.times[:frame+1]
    actual_vx = [v[0] for v in self.actual_velocities[:frame+1]]
    actual_vy = [v[1] for v in self.actual_velocities[:frame+1]]
    calc_vx = [v[0] for v in self.calculated_velocities[:frame+1]]
    calc_vy = [v[1] for v in self.calculated_velocities[:frame+1]]

    self.vx_actual.set_data(time, actual_vx)
    self.vy_actual.set_data(time, actual_vy)
    self.vx_calc.set_data(time, calc_vx)
    self.vy_calc.set_data(time, calc_vy)

    return self.line1, self.line2, self.joints
      
  def animate(self, points_history, times, actual_velocities, calculated_velocities):
    self.points_history = points_history
    self.times = times
    self.actual_velocities = actual_velocities
    self.calculated_velocities = calculated_velocities
    
    anim = FuncAnimation(self.fig, self.update,
                        frames=len(times), 
                        interval=50,
                        blit=True)
    
    plt.tight_layout()
    return HTML(anim.to_jshtml())

In [ ]:
# Initialize the verifier
verifier = Verifier2D()
points_history = []
times = []
actual_velocities = []
calc_velocities = []

# Time variables
time = 0
dt = 0.05
total_time = 1

# Initial points
angle0, angle5 = np.pi, np.pi/2
omega0, omega5 = -1, -1
prev_points, _ = legkinematics.calc_ang2pnt(angle0, angle5)

while time < total_time:
  # Update the angles
  angle0 = (angle0 + omega0 * dt) % (2*np.pi)
  angle5 = (angle5 + omega5 * dt) % (2*np.pi)

  # Forwards kinematics
  points, angles = legkinematics.calc_ang2pnt(angle0, angle5)

  calc_p3_velocity = legkinematics.calc_omg2vel(omega0, omega5, angles)
  actual_p3_velocity = (points[3] - prev_points[3]) / dt

  # Store data
  points_history.append(points)
  times.append(time)
  actual_velocities.append(actual_p3_velocity)
  calc_velocities.append(calc_p3_velocity)

  prev_points = points
  time += dt

animation = verifier.animate(points_history, times, actual_velocities, calc_velocities)
display(animation)

### Differential Inverse Kinematics

In [ ]:
def calc_vel2omg(self, vx3, vy3, point3):
  '''
  Calculate the angular velocity of the motors given the velocity and position of the foot
  '''
  angles, points = self.calc_pnt2ang(point3[0], point3[1])
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])

  i = -self.L1 * sin(angles[0]) - (self.L2 + self.L3) * sin(angles[1]) * (a*h-e*d) / (c*h-d*g)
  j = -sin(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g) 
  k = -self.L1 * cos(angles[0]) - (self.L2 + self.L3) * cos(angles[1]) * (a*h-e*d) / (c*h-d*g)
  l = -cos(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g)

  omega0 = (l * vx3 - j * vy3) / (i * l - j * k)
  omega5 = (-k * vx3 + i * vy3) / (i * l - j * k)

  return omega0, omega5

LegKinematics2D.calc_vel2omg = calc_vel2omg


Check if omg2vel and vel2omg are consistent

In [ ]:
legkinematics = LegKinematics2D()

# Given omegas and angles
omg0, omg5 = -0.5, -1
angle0, angle5 = 2*np.pi/3, np.pi/3

print("\nVelocity calculation")
print("Given Omega0: ", omg0, ", Omega5: ", omg5)
print("Given Angle0: ", angle0, ", Angle5: ", angle5)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5)
vx, vy = legkinematics.calc_omg2vel(omg0, omg5, angles)
print("Velocity: ", vx, vy)

print("\nAngular velocity calculation")
print("Given Velocity: ", vx, vy)
print("Given Point: ", points[3])
angle0, angle5 = legkinematics.calc_pnt2ang(points[3][0], points[3][1])
omega0, omega5 = legkinematics.calc_vel2omg(vx, vy, points[3])
print("Omega0: ", omega0, ", Omega5: ", omega5)

## Kinematics Analysis in $R^3$

Now, we will work on the 3D leg model, expanding the previous analysis to general 3D space.

In [ ]:
class LegKinematics(LegBase):
  def __init__(self):
    super().__init__()

### Forward Kinematics

In [ ]:
def calc_ang2pnt(self, angle0, angle5, angle6):
  '''
  Forward kinematics
  Given the angles of the motors, calculate the state of the leg, including the position of the points and the angles of the joints
  Return the points and angles of the joints
  '''
  # Start from 2D kinematics
  p0 = np.array([0, 0])
  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p5 = np.array([self.L0, 0])
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5
  L14 = np.linalg.norm(p4-p1)

  if(L14 > self.L4 + self.L2):
    print("Unreachable")
    return None
  elif (L14 == self.L4 + self.L2):
    print("Singularity")

  angle_214 = acos((self.L2**2 + L14**2 - self.L4**2) / (2 * self.L2 * L14))
  rotation_matrix1 = np.array([[np.cos(angle_214), -np.sin(angle_214)],
                              [np.sin(angle_214), np.cos(angle_214)]])
  rotation_matrix2 = np.array([[np.cos(-angle_214), -np.sin(-angle_214)],
                              [np.sin(-angle_214), np.cos(-angle_214)]])
  
  unit_vector_14 = (p4 - p1) / L14
  p2_1 = p1 + self.L2 * rotation_matrix1 @ unit_vector_14
  p2_2 = p1 + self.L2 * rotation_matrix2 @ unit_vector_14
  p2 = p2_1 if p2_1[1] < p2_2[1] else p2_2

  # Translate p2 to p3
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  # Store the 2D states
  points = np.array([p0, p1, p2, p3, p4, p5])

  # Translate points from 2D to 3D
  offset_angle = - angle6 # Rotate the current plane to the x-y plane
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(offset_angle), -sin(offset_angle)],
    [0, sin(offset_angle), cos(offset_angle)]
  ])
  points = np.concatenate([points, np.zeros((6, 1))], axis=1)
  points = points @ transformation_matrix

  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5,
    angle6
  ])

  return points, angles

LegKinematics.calc_ang2pnt = calc_ang2pnt

Verify if the forward kinematics works.

In [ ]:
legkinematics = LegKinematics()
N = 100  # Number of frames

# Define oscillating angles
angles6 = np.linspace(0, np.pi/2, N//2)
angles6 = np.concatenate([angles6, angles6[::-1]])  # Oscillate
angles0 = np.linspace(np.pi, np.pi/2, N)
angles5 = np.linspace(np.pi/2, 0, N)

# Create figure with two subplots
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# Set limits and labels for both subplots
for ax in [ax1, ax2]:
    ax.set_xlim3d([-15, 20])
    ax.set_ylim3d([-30, 5])
    ax.set_zlim3d([-15, 15])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

# Set different viewpoints
ax1.view_init(vertical_axis='y', azim=90)
ax2.view_init(vertical_axis='y', azim=30)

# Initialize empty lines for both subplots
line1_1, = ax1.plot([], [], [], 'blue', markersize=5)
line1_2, = ax1.plot([], [], [], 'blue', markersize=5)
scatter1 = ax1.scatter([], [], [], color='red', s=50, label='Joints')

line2_1, = ax2.plot([], [], [], 'blue', markersize=5)
line2_2, = ax2.plot([], [], [], 'blue', markersize=5)
scatter2 = ax2.scatter([], [], [], color='red', s=50, label='Joints')

# Animation update function
def update(frame):
    angle0 = angles0[frame]
    angle5 = angles5[frame]
    angle6 = angles6[frame]

    points, _ = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
    if points is not None:
        # Update first subplot (Side view)
        line1_1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
        line1_2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])
        line1_1.set_3d_properties(points[[0, 1, 2, 3], 2])
        line1_2.set_3d_properties(points[[0, 1, 2, 4, 5], 2])
        scatter1._offsets3d = (points[:, 0], points[:, 1], points[:, 2])

        # Update second subplot (Front view)
        line2_1.set_data(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1])
        line2_2.set_data(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1])
        line2_1.set_3d_properties(points[[0, 1, 2, 3], 2])
        line2_2.set_3d_properties(points[[0, 1, 2, 4, 5], 2])
        scatter2._offsets3d = (points[:, 0], points[:, 1], points[:, 2])

    return line1_1, line1_2, scatter1, line2_1, line2_2, scatter2

# Run animation
ani = FuncAnimation(fig, update, frames=N, interval=50)
HTML(ani.to_jshtml())

### Inverse Kinematics

In [98]:
def calc_pnt2ang(self, x, y, z):
  '''
  Calculate the inverse kinematics using the geometric method.
  Given the position of the foot, calculate the angles of the motors,
  Return the angles of the motors, which are p0 and p5.
  '''

  angle6 = atan2(-y, z) - np.pi/2
  if angle6 < 0:
    angle6 += np.pi

  # Translate the points from 3D to 2D
  offset_angle = angle6
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(offset_angle), -sin(offset_angle)],
    [0, sin(offset_angle), cos(offset_angle)]
  ])
  [x, y, z] = np.array([x, y, z]) @ transformation_matrix

  # Calculate the angle of the motor 0
  L03 = np.linalg.norm(np.array([x, y]))
  angle_305 = atan2(-y, x)
  if angle_305 < 0:
    angle_305 += np.pi
  angle_301 = acos((self.L1**2 + L03**2 - (self.L2 + self.L3)**2) / (2 * self.L1 * L03))
  angle0 = angle_305 + angle_301

  p1 = np.array([self.L1*cos(angle0), -self.L1*sin(angle0)])
  p2 = (np.array([x, y]) * self.L2 + p1 * self.L3) / (self.L2 + self.L3)
  p5 = np.array([self.L0, 0])

  # Calculate the angle of the motor 5
  L25 = np.linalg.norm(p2 - p5)
  angle_250 = atan2(-p2[1], self.L0 - p2[0])
  if angle_250 < 0:
    angle_250 += np.pi
  angle_254 = acos((L25**2 + self.L5**2 - self.L4**2) / (2 * L25 * self.L5))
  angle5 = np.pi - (angle_250 + angle_254)

  # Store the states
  p0 = np.array([0, 0])
  p3 = p2 + (p2-p1) / np.linalg.norm(p2-p1) * self.L3
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5

  angle1 = acos((p2-p1)[0] / np.linalg.norm(p2-p1))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))

  points = np.array([p0, p1, p2, p3, p4, p5])
  points = np.concatenate([points, np.zeros((6, 1))], axis=1)
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(-offset_angle), -sin(-offset_angle)],
    [0, sin(-offset_angle), cos(-offset_angle)]
  ])
  points = points @ transformation_matrix
  angles = np.array([
    angle0,
    angle1,
    None, # angle2
    None, # angle3
    angle4,
    angle5,
    angle6
  ])

  return angles, points

LegKinematics.calc_pnt2ang = calc_pnt2ang


Verify if the inverse kinematics consistent with the forward kinematics.

In [ ]:
# Given angles
angle0, angle5, angle6 = np.pi, np.pi/2, np.pi/4
print("Forward kinematics")
print("Given Angles: ", angle0, angle5, angle6)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
print("Points: ", points[3])

print("\nInverse kinematics")
print("Given Point: ", points[3])
angles, points = legkinematics.calc_pnt2ang(points[3][0], points[3][1], points[3][2])
print("Angles: ", angles[0], angles[5], angles[6])

### Differential Forward kinematics

In [23]:
def calc_omg2vel(self, omega0, omega5, omega6, angles, points):
  '''
  Calculate the velocity of the foot given the angular velocity of the motors
  '''
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])
  omega1 = omega0 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  vx2_2d = -self.L1 * omega0 * sin(angles[0]) - self.L2 * omega1 * sin(angles[1])
  vy2_2d = -self.L1 * omega0 * cos(angles[0]) - self.L2 * omega1 * cos(angles[1])

  vx3_2d = vx2_2d - self.L3 * omega1 * sin(angles[1])
  vy3_2d = vy2_2d - self.L3 * omega1 * cos(angles[1])

  vx3 = vx3_2d
  vy3 = vy3_2d * cos(angles[6]) + omega6 * np.sqrt(points[3][1]**2 + points[3][2]**2) * sin(angles[6])
  vz3 = vy3_2d * sin(angles[6]) - omega6 * np.sqrt(points[3][1]**2 + points[3][2]**2) * cos(angles[6])
  
  return vx3, vy3, vz3

LegKinematics.calc_omg2vel = calc_omg2vel


Verify if the differential forward kinematics consistent with the forward kinematics.

In [24]:
class Verifier:
  def __init__(self):
    # Create figure with two subplots
    self.fig = plt.figure(figsize=(15, 5))
        
    # Create 3D subplot for mechanism
    self.ax_mech = self.fig.add_subplot(121, projection='3d')
    
    # Set up axes
    self.ax_mech.set_xlabel("X")
    self.ax_mech.set_ylabel("Y")
    self.ax_mech.set_zlabel("Z")
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_zlim(-10, 10)
    self.ax_mech.set_box_aspect([35, 35, 20])
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')

    # Create 2D subplot for velocity
    self.ax_vel = self.fig.add_subplot(122)
    self.ax_vel.set_xlim(0, 2)
    self.ax_vel.set_ylim(-20, 20)
    self.ax_vel.grid(True)
    self.ax_vel.set_title('P3 Velocity Components')
    self.ax_vel.set_xlabel('Time (s)')
    self.ax_vel.set_ylabel('Velocity')

    # Data storage
    self.times = []
    self.actual_velocities = []
    self.calculated_velocities = []
    self.points_history = []

    # Dummy line for animation update
    self.dummyLine, = self.ax_mech.plot([], [], [])

    # Initialize velocity plot lines
    self.vx_actual, = self.ax_vel.plot([], [], label='Vx')
    self.vy_actual, = self.ax_vel.plot([], [], label='Vy')
    self.vz_actual, = self.ax_vel.plot([], [], label='Vz')
    self.vx_calc, = self.ax_vel.plot([], [], label='Vx Calc')
    self.vy_calc, = self.ax_vel.plot([], [], label='Vy Calc')
    self.vz_calc, = self.ax_vel.plot([], [], label='Vz Calc')
    
    self.ax_mech.view_init(vertical_axis='y', azim=30)
    self.ax_vel.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

  def update(self, frame):
    points = self.points_history[frame]

    self.ax_mech.cla()
    self.ax_mech.set_xlabel("X")
    self.ax_mech.set_ylabel("Y")
    self.ax_mech.set_zlabel("Z")
    self.ax_mech.set_xlim(-15, 20)
    self.ax_mech.set_ylim(-30, 5)
    self.ax_mech.set_zlim(-10, 10)
    self.ax_mech.set_box_aspect([35, 35, 20])
    self.ax_mech.grid(True)
    self.ax_mech.set_title('Leg Mechanism')
    
    # Update chain
    self.ax_mech.plot3D(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1], points[[0, 1, 2, 3], 2], 'blue')
    self.ax_mech.plot3D(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1], points[[0, 1, 2, 4, 5], 2], 'blue')
    self.ax_mech.scatter3D(points[:, 0], points[:, 1], points[:, 2], c='r', marker='o')
    
    # Update velocity vector at p3
    if frame > 0:
      actual_vel = self.actual_velocities[frame]
      calc_vel = self.calculated_velocities[frame]

      # Plot velocity vector
      self.ax_mech.quiver(points[3, 0], points[3, 1], points[3, 2], actual_vel[0], actual_vel[1], actual_vel[2], color='g', label='V Actual')
      self.ax_mech.quiver(points[3, 0], points[3, 1], points[3, 2], calc_vel[0], calc_vel[1], calc_vel[2], color='r', label='V Calc')
        
    # Plot velocity components over time
    time = self.times[:frame+1]
    actual_vx = [v[0] for v in self.actual_velocities[:frame+1]]
    actual_vy = [v[1] for v in self.actual_velocities[:frame+1]]
    actual_vz = [v[2] for v in self.actual_velocities[:frame+1]]
    calc_vx = [v[0] for v in self.calculated_velocities[:frame+1]]
    calc_vy = [v[1] for v in self.calculated_velocities[:frame+1]]
    calc_vz = [v[2] for v in self.calculated_velocities[:frame+1]]

    self.vx_actual.set_data(time, actual_vx)
    self.vy_actual.set_data(time, actual_vy)
    self.vz_actual.set_data(time, actual_vz)
    self.vx_calc.set_data(time, calc_vx)
    self.vy_calc.set_data(time, calc_vy)
    self.vz_calc.set_data(time, calc_vz)

    return self.dummyLine,
      
  def animate(self, points_history, times, actual_velocities, calculated_velocities):
    self.points_history = points_history
    self.times = times
    self.actual_velocities = actual_velocities
    self.calculated_velocities = calculated_velocities
    
    anim = FuncAnimation(self.fig, self.update,
                        frames=len(times), 
                        interval=50,
                        blit=True)
    
    plt.tight_layout()
    return HTML(anim.to_jshtml())

In [ ]:
# Initialize the verifier
verifier = Verifier()
points_history = []
times = []
actual_velocities = []
calc_velocities = []

# Time variables
time = 0
dt = 0.02
total_time = 1

# Initial points
angle0, angle5, angle6 = np.pi, np.pi/2, 0
omega0, omega5, omega6 = -1, -0.5, 0.5
prev_points, _ = legkinematics.calc_ang2pnt(angle0, angle5, angle6)

while time < total_time:
  # Update the angles
  angle0 = (angle0 + omega0 * dt) % (2*np.pi)
  angle5 = (angle5 + omega5 * dt) % (2*np.pi)
  angle6 = (angle6 + omega6 * dt) % (2*np.pi)

  # Forwards kinematics
  points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
  calc_p3_velocity = legkinematics.calc_omg2vel(omega0, omega5, omega6, angles, points)
  actual_p3_velocity = (points[3] - prev_points[3]) / dt

  # Store data
  points_history.append(points)
  times.append(time)
  actual_velocities.append(actual_p3_velocity)
  calc_velocities.append(calc_p3_velocity)

  prev_points = points
  time += dt

animation = verifier.animate(points_history, times, actual_velocities, calc_velocities)
display(animation)

### Differential Inverse kinematics

In [26]:
def calc_vel2omg(self, vx3, vy3, vz3, points, angles):
  '''
  Calculate the angular velocity of the motors given the velocity and position of the foot
  '''
  # Convert the velocity to 2D
  vx3_2d = vx3
  vy3_2d = vy3 * cos(angles[6]) + vz3 * sin(angles[6])
  v_omega6 = vy3 * sin(angles[6]) - vz3 * cos(angles[6])
  omega6 = v_omega6 / np.sqrt(points[3][1]**2 + points[3][2]**2)
  
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])

  i = -self.L1 * sin(angles[0]) - (self.L2 + self.L3) * sin(angles[1]) * (a*h-e*d) / (c*h-d*g)
  j = -sin(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g) 
  k = -self.L1 * cos(angles[0]) - (self.L2 + self.L3) * cos(angles[1]) * (a*h-e*d) / (c*h-d*g)
  l = -cos(angles[1]) * (self.L2 + self.L3) * (b*h-f*d) / (c*h-d*g)

  omega0 = (l * vx3_2d - j * vy3_2d) / (i * l - j * k)
  omega5 = (-k * vx3_2d + i * vy3_2d) / (i * l - j * k)

  return omega0, omega5, omega6

LegKinematics.calc_vel2omg = calc_vel2omg


Verify if the differential inverse kinematics consistent with the differential forward kinematics.

In [ ]:
omega0, omega5, omega6 = -1, -1, 1
angle0, angle5, angle6 = np.pi, np.pi/2, np.pi/4
print("\nVelocity calculation")
print("Given Omega0: ", omega0, ", Omega5: ", omega5, ", Omega6: ", omega6)
print("Given Angle0: ", angle0, ", Angle5: ", angle5, ", Angle6: ", angle6)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
vx, vy, vz = legkinematics.calc_omg2vel(omega0, omega5, omega6, angles, points)
print("End Point: ", points[3])
print("Velocity: ", vx, vy, vz)

print("\nAngular velocity calculation")
print("Given Velocity: ", vx, vy, vz)
print("Given Point: ", points[3])
angles, points = legkinematics.calc_pnt2ang(points[3][0], points[3][1], points[3][2])
omega0, omega5, omega6 = legkinematics.calc_vel2omg(vx, vy, vz, points, angles)
print("Angle0: ", angles[0], ", Angle5: ", angles[5], ", Angle6: ", angles[6])
print("Omega0: ", omega0, ", Omega5: ", omega5, ", Omega6: ", omega6)

## DH method

In this section, we will redo the kinematics analysis using the DH method and the Jacobian matrix.

### Denavit-Hartenberg(DH) method

$ a_i $: distance between $ O_{i-1} $ and $ O_i $ measured along $ x_{i-1} $  
$ d_i $: distance between $ O_{i-1} $ and $ O_i $ measured along $ z_{i-1} $  
$ \alpha_i $: angle between $ z_{i-1} $ and $ z_i $ measured about $ x_i $ when the rotation is made counterclockwise.  
$ \theta_i $: angle between $ x_{i-1} $ and $ x_i $ measured about $ z_{i-1} $ when the rotation is made counterclockwise.  

### Reference

[YT video - 2.13 D-H parameters of 4-bar parallel robot](https://www.youtube.com/watch?v=oIz5L0VpLQo&t=332s)  
[YT video - Introduction to DH Convention](https://www.youtube.com/watch?v=JSEPoyIypHQ)

### Transformation matrix

$$
T = \begin{bmatrix}
\cos(\theta) & -\sin(\theta)\cos(\alpha) & \sin(\theta)\sin(\alpha)  & a\cos(\theta) \\
\sin(\theta) & \cos(\theta)\cos(\alpha) & -\cos(\theta)\sin(\alpha) & a\sin(\theta) \\
0 & \sin(\alpha) & \cos(\alpha) & d \\
0 & 0 & 0 & 1
\end{bmatrix}

 = \begin{bmatrix}
R_{11} & R_{12} & R_{13} & p_x \\
R_{21} & R_{22} & R_{23} & p_y \\
R_{31} & R_{32} & R_{33} & p_z \\
0 & 0 & 0 & 1
\end{bmatrix}

= 
\begin{bmatrix}
  R_{3 \times 3} & p_{3 \times 1} \\
  0 & 1
\end{bmatrix}
$$

In [28]:
def get_coordinate_transform(self, theta, d, a, alpha):
  return np.array([[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
                   [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
                   [0, np.sin(alpha), np.cos(alpha), d],
                   [0, 0, 0, 1]])

LegKinematics.get_coordinate_transform = get_coordinate_transform

In [29]:
def get_transform_matrix(self, angles):
  # Base to joint0
  Ab0_1 = self.get_coordinate_transform(theta=-np.pi/2, d=0, a=0,  alpha=0)
  Ab0_2 = self.get_coordinate_transform(theta=0, d=0, a=0,  alpha=-np.pi/2)
  Ab0_3 = self.get_coordinate_transform(theta=angles[0], d=0, a=0,  alpha=0)
 
  # Joint0 to joint1
  A01_1 = self.get_coordinate_transform(theta=0, d=0, a=0, alpha=np.pi/2)
  A01_2 = self.get_coordinate_transform(theta=np.pi/2-angles[1], d=0, a=0, alpha=0)

  # Joint 1 to joint 2
  A12_1 = self.get_coordinate_transform(theta=0, d=0, a=self.L1, alpha=0)
  A12_2 = self.get_coordinate_transform(theta=angles[1]-angles[2], d=0, a=0, alpha=0)

  # Joint 2 to end effector
  A2e = self.get_coordinate_transform(theta=0, d=0, a=self.L2+self.L3,  alpha=0)

  T0 = Ab0_1 @ Ab0_2 @ Ab0_3
  T1 = T0 @ A01_1 @ A01_2
  T2 = T1 @ A12_1 @ A12_2
  Te = T2 @ A2e

  T = [T0, T1, T2, Te]
  return T

LegKinematics.get_transform_matrix = get_transform_matrix


In [ ]:
angle0, angle5, angle6 = np.pi, np.pi/2, 0
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
angles = np.array([angles[6], angles[0], angles[1]])
T = legkinematics.get_transform_matrix(angles)

base = np.array([0, 0, 0, 1])
joint0 = T[0] @ base
joint1 = T[1] @ base
joint2 = T[2] @ base
end_effector = T[3] @ base

# Extract rotation matrix
R0 = T[0][:3, :3]
R1 = T[1][:3, :3]
R2 = T[2][:3, :3]
Re = T[3][:3, :3]

print("\nX axis direction of each frame in base frame: ")
x = np.array([1, 0, 0, 1])
print("Base: ", x)
print("Joint0: ", T[0] @ x)
print("Joint1: ", T[1] @ x)
print("Joint2: ", T[2] @ x)
print("End effector: ", T[3] @ x)


Verify if the transformation matrix fits the forward kinematics.

In [ ]:
angle0, angle5, angle6 = np.pi, np.pi/2, 0
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
# print("Points: ")
# for p in points:
  # print(p)

angles = np.array([angles[6], angles[0], angles[1]])
T = legkinematics.get_transform_matrix(angles)
print("\nTransform Matrix")
for t in T:
  print(t[:3, 3])

### Jacobian matrix

$$
v = J \times \dot{q}
$$

$$
\dot{q} = J^{-1} \times v
$$

$$
J = \begin{bmatrix}
J_{pi} \\
J_{oi}
\end{bmatrix}

= \begin{bmatrix}
z_{i-1} \times (p_n - p_{i-1}) \\
z_{i-1}
\end{bmatrix}
$$
for revolute joint

In [32]:
def get_jacobian(self, T_list):
        T0, T1, T2, Te = T_list

        # Extract positions (translation vectors)
        p = [T[:3, 3] for T in T_list]  # p0, p1, p2, p3, p4, p5, pe
        pe = p[-1]  # End-effector position

        # Extract rotation matrices and z-axes
        R = [T[:3, :3] for T in T_list]
        z = [R_i[:, 2] for R_i in R]  # z-axis is third column (DH convention)

        # Active joints: 0 (theta0), 1 (theta1), 5 (theta5)
        active_joints = [0, 1, 2]  # Indices of motor-driven joints
        J = np.zeros((6, 3))  # 6 rows (vx, vy, vz, wx, wy, wz), 3 columns

        # Compute Jacobian columns
        for i, joint_idx in enumerate(active_joints):
            z_i = z[joint_idx]  # Rotation axis in base frame
            p_i = p[joint_idx]  # Position of joint frame
            J[0:3, i] = np.cross(z_i, pe - p_i)  # Linear velocity contribution
            J[3:6, i] = z_i  # Angular velocity contribution

        return J

LegKinematics.get_jacobian = get_jacobian

In [33]:
def calc_omega1(self, omega0, omega5, angles):
  '''
  Calculate the angular velocity of the motor 1 given the angular velocities of the motors 0 and 5
  '''
  a = self.L1 * sin(angles[0])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[1])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[0])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[1])
  h = self.L4 * cos(angles[4])
  omega1 = omega0 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  return omega1

LegKinematics.calc_omega1 = calc_omega1

Verify if the Jacobian matrix fits the differential forward kinematics.

In [ ]:
angle0, angle5, angle6 = np.pi, np.pi/2, 0
omega0, omega5, omega6 = 1, 1, 0
print("Given Angles: ", angle0, angle5, angle6)
print("Given Omegas: ", omega0, omega5, omega6)
points, angles = legkinematics.calc_ang2pnt(angle0, angle5, angle6)
vx, vy, vz = legkinematics.calc_omg2vel(omega0, omega5, omega6, angles, points)
print("\nEnd Point: ", points[3])
print("Velocity: ", vx, vy, vz)

T = legkinematics.get_transform_matrix(np.array([angle6, angle0, angles[1]]))
print("\nT[4] position:", T[3][0:3, 3])
J = legkinematics.get_jacobian(T)
omega1 = legkinematics.calc_omega1(omega0, omega5, angles)
vel = J @ np.array([omega6, omega0, omega1])
print("Vel from Jacobian: ", vel)


## Improve the function 

Forward kinematics
```python
def calc_ang2pnt(angles):
  # Input: [angle0, angle1, angle5], motor_angles, a list of 3 angles of the motors
  # Output: [x, y, z], the position of the end effector
```

Inverse kinematics
```python
def calc_pnt2ang(point):
  # Input: [x, y, z], the position of the end effector
  # Output: [angle0, angle1, angle5], a list of 3 angles of the motors
```

Differential forward kinematics
```python
def calc_omg2vel(omegas):
  # Input: [omega0, omega1, omega5], the angular velocity of the motors
  # Output: [vx, vy, vz], the linear velocity of the end effector
```



### Parameters and Objects

In [51]:
import numpy as np

class LegState:
  def __init__(self, points=None, angles=None):
    self.points = points
    self.angles = angles
    self.velocities = None
    self.omegas = None

    # Alternative state result from the inverse kinematics
    self.points_alt = None
    self.angles_alt = None

class Leg:
  def __init__(self):
    self.L0 = 8.16
    self.L1 = 8.0
    self.L2 = 10.0
    self.L3 = 8.0
    self.L4 = 13.0
    self.L5 = 8.0
    self.state = LegState()


### Plot the leg

In [36]:
def plot_leg(self, state=None):
  '''
  Plot the leg in 3D
  '''
  state = self.state if state is None else state
  fig = plt.figure(figsize=(6, 6))
  ax = fig.add_subplot(111, projection='3d')

  # Plot the leg
  points = state.points
  ax.plot(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1], points[[0, 1, 2, 3], 2], 'blue')
  ax.plot(points[[0, 1, 2, 4, 5], 0], points[[0, 1, 2, 4, 5], 1], points[[0, 1, 2, 4, 5], 2], 'blue')
  ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='r', marker='o')

  # Set labels
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')

  # Set limits
  ax.set_xlim([-15, 20])
  ax.set_ylim([-30, 5])
  ax.set_zlim([-15, 15])

  ax.view_init(vertical_axis='y', azim=30)

  plt.show()

Leg.plot_leg = plot_leg


### Forward Kinematics

In [59]:
def calc_ang2pnt(self, motor_angles):
  '''
  Forward kinematics
  Given the angles of the motors, calculate the end point of the leg and the state of the leg.

  @param motor_angles: Angles of the motors, size 3 [angle0, angle1, angle5]
  @return: state object
  '''
  angle0, angle1, angle5 = motor_angles

  # Start from 2D kinematics
  p1 = np.array([0, 0])
  p2 = np.array([self.L1*cos(angle1), -self.L1*sin(angle1)])
  p5 = np.array([self.L0, 0])
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5
  
  L14 = np.linalg.norm(p4-p1)
  L24 = np.linalg.norm(p4-p2)

  # Check if the leg is reachable
  if(L24 > self.L4 + self.L2):
    print("Unreachable")
    return None
  elif (L24 == self.L4 + self.L2):
    print("Singularity")

  # Calculate angle2
  angle_324 = acos((self.L2**2 + L24**2 - self.L4**2) / (2 * self.L2 * L24))
  angle_421 = acos((self.L1**2 + L24**2 - L14**2) / (2 * self.L1 * L24))
  angle2 = angle_324 + angle_421 - (np.pi - angle1)

  # Calculate pe and p3 by averaging the positions
  pe = np.array([(self.L2+self.L3)*cos(angle2), -(self.L2+self.L3)*sin(angle2)]) + p2
  p3 = (pe*self.L2+p2*self.L3)/(self.L2+self.L3)
  angle4 = acos((p3-p4)[0] / np.linalg.norm(p3-p4))

  angles = np.array([angle0, angle1, angle2, None, angle4, angle5])
  points = np.array([p1, p2, p3, pe, p4, p5])

  # Translate points from 2D to 3D, rotate the current plane to the x-y plane
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(-angle0), -sin(-angle0)],
    [0, sin(-angle0), cos(-angle0)]
  ])
  points = np.concatenate([points, np.zeros((points.shape[0], 1))], axis=1)
  points = points @ transformation_matrix

  # Store to the state object
  self.state.angles = angles
  self.state.points = points

  return points[3]

Leg.calc_ang2pnt = calc_ang2pnt


### Inverse Kinematics

In [63]:
def calc_pnt2ang(self, end_effector_point):
  '''
  Inverse kinematics
  Given the end point of the leg, calculate the angles of the motors and the state of the leg.

  @param end_point: Position of the end point, size 3 [x, y, z]
  @return: state object
  '''
  x, y, z = end_effector_point

  # Calculate the angle of the motor 0
  angle0 = -atan2(z, -y)

  # Translate the points from 3D to 2D
  offset_angle = -angle0
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(offset_angle), -sin(offset_angle)],
    [0, sin(offset_angle), cos(offset_angle)]
  ])
  [x, y, z] = transformation_matrix @ np.array([x, y, z])
  print("x, y, z: ", x, y, z)
  pe_2d = np.array([x, y])

  # Angle 1
  L1e = np.linalg.norm(pe_2d)
  angle_e15 = atan2(-y, x)
  if angle_e15 < 0:
    angle_e15 += np.pi
  angle_e12 = acos((self.L1**2 + L1e**2 - (self.L2 + self.L3)**2) / (2 * self.L1 * L1e))
  angle1 = angle_e15 + angle_e12

  # Points based on angle 1
  p1 = np.array([0, 0])  
  p2 = np.array([self.L1*cos(angle1), -self.L1*sin(angle1)])
  p3 = (pe_2d * self.L2 + p2 * self.L3) / (self.L2 + self.L3)
  p5 = np.array([self.L0, 0])

  # Angle 5
  L35 = np.linalg.norm(p3 - p5)
  angle_350 = atan2(-p3[1], self.L0 - p3[0])
  if angle_350 < 0:
    angle_350 += np.pi
  angle_354 = acos((L35**2 + self.L5**2 - self.L4**2) / (2 * L35 * self.L5))
  angle5 = np.pi - (angle_350 + angle_354)

  # Point 4
  p4 = np.array([self.L5*cos(angle5), -self.L5*sin(angle5)]) + p5

  # Alt Angle 1
  angle1_alt = angle_e15 - angle_e12

  # Alt Points based on angle 1
  p2_alt = np.array([self.L1*cos(angle1_alt), -self.L1*sin(angle1_alt)])
  p3_alt = (pe_2d * self.L2 + p2_alt * self.L3) / (self.L2 + self.L3)

  # Alt Angle 5
  L35_alt = np.linalg.norm(p3_alt - p5)
  angle_350_alt = atan2(-p3_alt[1], self.L0 - p3_alt[0])
  if angle_350_alt < 0:
    angle_350_alt += np.pi
  angle_354_alt = acos((L35_alt**2 + self.L5**2 - self.L4**2) / (2 * L35_alt * self.L5))
  angle5_alt = np.pi - (angle_350_alt + angle_354_alt)

  # Alt Point 4
  p4_alt = np.array([self.L5*cos(angle5_alt), -self.L5*sin(angle5_alt)]) + p5

  # Store the states
  angle2 = acos((pe_2d-p2)[0] / np.linalg.norm(pe_2d-p2))
  angle4 = acos((p2-p4)[0] / np.linalg.norm(p2-p4))
  angles = np.array([angle0, angle1, angle2, None, angle4, angle5])

  angle2_alt = acos((pe_2d-p2_alt)[0] / np.linalg.norm(pe_2d-p2_alt))
  angle4_alt = acos((p2_alt-p4_alt)[0] / np.linalg.norm(p2_alt-p4_alt))
  angles_alt = np.array([angle0, angle1_alt, angle2_alt, None, angle4_alt, angle5_alt])

  # Translate the points from 2D to 3D
  points = np.array([p1, p2, p3, pe_2d, p4, p5])
  points_alt = np.array([p1, p2_alt, p3_alt, pe_2d, p4_alt, p5])
  points = np.concatenate([points, np.zeros((points.shape[0], 1))], axis=1)
  points_alt = np.concatenate([points_alt, np.zeros((points_alt.shape[0], 1))], axis=1)
  transformation_matrix = np.array([
    [1, 0, 0],
    [0, cos(-angle0), -sin(-angle0)],
    [0, sin(-angle0), cos(-angle0)]
  ])
  points = points @ transformation_matrix
  points_alt = points_alt @ transformation_matrix

  self.state.points = points
  self.state.angles = angles
  self.state.points_alt = points_alt
  self.state.angles_alt = angles_alt

  return [angle0, angle1, angle5]

Leg.calc_pnt2ang = calc_pnt2ang


In [ ]:
motor_angles = np.array([np.pi, np.pi, np.pi/2])
leg = Leg()
print("Forward kinematics")
print("Given angles", motor_angles)
pe = leg.calc_ang2pnt(motor_angles)
# leg.plot_leg()
print("End Point: ", pe)

print("\nInverse kinematics")
print("Given Point: ", pe)
angles = leg.calc_pnt2ang(pe)
print(leg.state.points[3])
# leg.plot_leg()
print("Angles: ", angles)

### Differential Forward Kinematics

[Modern Robotics, Chapter 6.2: Numerical Inverse Kinematics (Part 1 of 2)
](https://www.youtube.com/watch?v=VhUA0jf7tI8)

In [40]:
def numerical_jacobian(self, motor_angles, delta=1e-4):
  pe = self.calc_ang2pnt(motor_angles)
  J = np.zeros((6, 3))
  for i in range(3):
    angles_perturbed = motor_angles.copy()
    angles_perturbed[i] += delta
    pe_perturbed = self.calc_ang2pnt(angles_perturbed)
    J[0:3, i] = (pe_perturbed - pe) / delta
    J[3:6, i] = [1 if i == 0 else 0, 0, 1 if i != 0 else 0] # Angular part simplified (adjust based on axes)
  return J

Leg.numerical_jacobian = numerical_jacobian

In [41]:
def calc_omg2vel(self, motor_omegas):
  '''
  Calculate the velocity of the foot given the angular velocity of the motors
  '''
  omega0, omega1, omega5 = motor_omegas
  angles = self.state.angles
  points = self.state.points

  a = self.L1 * sin(angles[1])
  b = -self.L5 * sin(angles[5])
  c = -self.L2 * sin(angles[2])
  d = self.L4 * sin(angles[4])
  e = self.L1 * cos(angles[1])
  f = -self.L5 * cos(angles[5])
  g = -self.L2 * cos(angles[2])
  h = self.L4 * cos(angles[4])
  omega2 = omega1 * (a*h-e*d) / (c*h-d*g) + omega5 * (b*h-f*d) / (c*h-d*g)

  vx2_2d = -self.L1 * omega1 * sin(angles[1]) - self.L2 * omega2 * sin(angles[2])
  vy2_2d = -self.L1 * omega1 * cos(angles[1]) - self.L2 * omega2 * cos(angles[2])

  vx3_2d = vx2_2d - self.L3 * omega2 * sin(angles[2])
  vy3_2d = vy2_2d - self.L3 * omega2 * cos(angles[2])

  vx3 = vx3_2d
  vy3 = vy3_2d * cos(angles[0]) + omega0 * np.sqrt(points[3][1]**2 + points[3][2]**2) * sin(angles[0])
  vz3 = vy3_2d * sin(angles[0]) - omega0 * np.sqrt(points[3][1]**2 + points[3][2]**2) * cos(angles[0])

  vel = np.array([vx3, vy3, vz3])
  return vel

Leg.calc_omg2vel = calc_omg2vel


Verify

In [ ]:
angles = np.array([0, 3*np.pi/4, np.pi/4])
omegas = np.array([1, -1, 1])
leg = Leg()
print("Given Angles: ", angles)
print("Given Omegas: ", omegas)

# Improved param
end_point = leg.calc_ang2pnt(angles)
vel = leg.calc_omg2vel(omegas)
print("\nEnd Point: ", end_point)
print("Velocity: ", vel)

# Jacobian
J = leg.numerical_jacobian(angles)
vel = J @ omegas
print("\nVelocity from Jacobian: ", vel)

J_inv = np.linalg.pinv(J)
omg = J_inv @ vel
print("Omegas from Jacobian: ", omg)

# Origianl
legkinematics = LegKinematics()
angles, points = legkinematics.calc_pnt2ang(end_point[0], end_point[1], end_point[2])
vx, vy, vz = legkinematics.calc_omg2vel(omegas[1], omegas[2], omegas[0], angles, points)
omega0, omega1, omega5 = legkinematics.calc_vel2omg(vx, vy, vz, points, angles)
print("\nOriginal Velocity: ", vx, vy, vz)
print("Original Omegas: ", omega0, omega1, omega5)



## Body inverse kinematics

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from src.leg import Leg

### Transformation matrix

[Coordinate Transformations](https://motion.cs.illinois.edu/RoboticSystems/CoordinateTransformations.html)

Rigid transformation

$$
T_s = T_b T_s^b
$$
(s->w) = (b->w) (s->b)

$$
p^b = T_s^b p^s
$$

In [2]:
class QuadModel:
  def __init__(self):
    # Body dimensions, shoulder on the vertices
    body_width = 20
    body_length = 40
    self.body_z = 17

    # Foothold coordinates in world frame
    self.footholds = [
      np.array([body_length/2-4, (body_width/2), 0]),  # Front-left
      np.array([body_length/2-4, -(body_width/2), 0]), # Front-right
      np.array([-body_length/2+4, (body_width/2), 0]), # Back-left
      np.array([-body_length/2+4, -(body_width/2), 0]) # Back-right
    ]

    # Shoulder coordinates in body frame
    self.shoulders = [
      np.array([body_length/2, body_width/2, 0]),  # Front-left
      np.array([body_length/2, -body_width/2, 0]), # Front-right
      np.array([-body_length/2, body_width/2, 0]), # Back-left
      np.array([-body_length/2, -body_width/2, 0]) # Back-right
    ]

    # Transformation matrix
    # Naming convention:
    #   b: body frame
    #   s: shoulder frame

    # Body to World transformation
    self.Tb = self.calc_transform_matrix([0, 0, 0], [0, 0, self.body_z])
    
    # Rotation for shoulder frames to align with the body frame
    Rbs_front = self.calc_rotation_matrix(0, -np.pi, 0) @ self.calc_rotation_matrix(-np.pi/2, 0, 0)
    Rbs_back = self.calc_rotation_matrix(np.pi/2, 0, 0)

    # Shoulder to Body transformation, remain constant all the time
    self.Tbs_lst = []
    for shoulder in self.shoulders:
      Tbs = np.eye(4)
      Tbs[:3, :3] = Rbs_front if shoulder[0] > 0 else Rbs_back
      Tbs[:3, 3] = shoulder
      self.Tbs_lst.append(Tbs)

    # Initialize the legs and calculate the end effector points
    self.legs = [Leg() for _ in range(4)]
    for (i, foothold) in enumerate(self.footholds):
      Ts = self.Tb @ self.Tbs_lst[i]
      Ts_inv = self.calc_inv_T(Ts)
      pe = self.calc_projected_point(Ts_inv, foothold)
      self.legs[i].ee_point = pe

  def calc_rotation_matrix(self, roll, pitch, yaw):
    """
    Compute 3D rotation matrix from roll, pitch, yaw (XYZ Euler angles).
    """
    Rx = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])
    
    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])
    
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    return Rz @ Ry @ Rx

  def calc_transform_matrix(self, orientation, position):
    """
    Compute 4x4 transformation matrix from roll, pitch, yaw (XYZ Euler angles) and position for 
    current frame to align with the target frame. 

    @param orientation: Roll, pitch, yaw angles for current frame to align with the target frame
    @param position: Position of the current frame's origin in the target frame's coordinates
    """
    [roll, pitch, yaw] = orientation
    R = self.calc_rotation_matrix(roll, pitch, yaw)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = position
    return T
  
  def calc_inv_T(self, T):
    """
    Compute the inverse of the transformation matrix.
    """
    R = T[:3, :3]
    p = T[:3, 3]

    R_inv = R.T
    p_inv = -R_inv @ p

    T_inv = np.eye(4)
    T_inv[:3, :3] = R_inv
    T_inv[:3, 3] = p_inv

    return T_inv

  def calc_projected_point(self, transform_matrix, point):
    """
    Project a point to the new frame using the transformation matrix.

    @param transform_matrix: 4x4 transformation matrix
    @param point: Point to be projected
    """
    point = np.append(point, 1)
    projected_point = transform_matrix @ point
    return projected_point[:3]
  

In [3]:
def update_body_pose(self, roll, pitch, yaw):
  """
  Update the orientation and position of the body
  """
  
  # Body to World transformation
  self.Tb = self.calc_transform_matrix([roll, pitch, yaw], [0, 0, self.body_z])
  
  # Shoulder to World transformation
  for (i, foothold) in enumerate(self.footholds):
    Ts = self.Tb @ self.Tbs_lst[i]
    Ts_inv = self.calc_inv_T(Ts)
    pe = self.calc_projected_point(Ts_inv, foothold)
    self.legs[i].ee_point = pe

QuadModel.update_body_pose = update_body_pose


In [4]:
def plot_transform_list(self, ax, axis_length=5.0):
    """
    Plot a list of transformation matrices in 3D.
    
    Args:
        T_list: list of 4x4 numpy arrays
        titles: list of str, labels for each frame
        axis_length: float, length of axes
    """

    titles = ["Front-Right", "Front-Left", "Rear-Right", "Rear-Left"]
    
    Ts_lst = []
    for i, (Tbs, title) in enumerate(zip(self.Tbs_lst, titles)):
        Ts = self.Tb @ Tbs
        p = Ts[:3, 3]
        R = Ts[:3, :3]
        Ts_lst.append(Ts)

        # Define axes
        x_axis = R @ np.array([axis_length, 0, 0])
        y_axis = R @ np.array([0, axis_length, 0])
        z_axis = R @ np.array([0, 0, axis_length])

        # Plot shoulder
        ax.scatter([p[0]], [p[1]], [p[2]], s=100, label=title)

        # Plot the leg
        points_world = []
        for point in self.legs[i].points:
            point_world = self.calc_projected_point(Ts, point)
            points_world.append(point_world)
        points = np.array(points_world)
        ax.plot3D(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1], points[[0, 1, 2, 3], 2], 'blue')
        ax.plot3D(points[[2, 4, 5], 0], points[[2, 4, 5], 1], points[[2, 4, 5], 2], 'blue')

        # Plot axes
        ax.quiver(p[0], p[1], p[2], x_axis[0], x_axis[1], x_axis[2], 
                  color='red', linewidth=1)
        ax.quiver(p[0], p[1], p[2], y_axis[0], y_axis[1], y_axis[2], 
                  color='green', linewidth=1)
        ax.quiver(p[0], p[1], p[2], z_axis[0], z_axis[1], z_axis[2], 
                  color='blue', linewidth=1)
        
        # Plot the desired footholds
        ax.scatter([self.footholds[i][0]], [self.footholds[i][1]], [self.footholds[i][2]], 
                       color='black', s=50)
        
    shoulders = np.array([T[:3, 3] for T in Ts_lst])
    ax.plot3D(shoulders[[0, 1, 3, 2, 0], 0], shoulders[[0, 1, 3, 2, 0], 1], shoulders[[0, 1, 3, 2, 0], 2], 'blue')

    
    # Set labels and limits
    ax.set_xlabel('X (cm)')
    ax.set_ylabel('Y (cm)')
    ax.set_zlabel('Z (cm)')
    ax.set_title("Body Inverse Kinematics")

    ax.set_xlim((-30, 30))
    ax.set_ylim((-20, 20))
    ax.set_zlim((0, 20))
    ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio (1:1:1)
    ax.view_init(azim=40)
    ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

QuadModel.plot_transform_list = plot_transform_list


### Verify the body inverse kinematics using sliders

In [ ]:
# Setup plot and sliders
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(121, projection='3d')
plt.subplots_adjust(bottom=0.25)  # Adjust bottom to fit sliders

robot = QuadModel()
robot.update_body_pose(0.2, 0, 0)
robot.plot_transform_list(ax)

# Add sliders
ax_roll = plt.axes([0.25, 0.1, 0.3, 0.03])
ax_pitch = plt.axes([0.25, 0.05, 0.3, 0.03])
ax_yaw = plt.axes([0.25, 0.0, 0.3, 0.03])

roll_slider = Slider(ax_roll, 'Roll', -np.pi*0.08, np.pi*0.08, valinit=0)
pitch_slider = Slider(ax_pitch, 'Pitch', -np.pi*0.08, np.pi*0.08, valinit=0)
yaw_slider = Slider(ax_yaw, 'Yaw', -np.pi*0.08, np.pi*0.08, valinit=0)

def update(val):
    ax.cla()
    roll = roll_slider.val
    pitch = pitch_slider.val
    yaw = yaw_slider.val

    robot.update_body_pose(roll, pitch, yaw)
    robot.plot_transform_list(ax)

    ax.set_xlim((-30, 30))
    ax.set_ylim((-20, 20))
    ax.set_zlim((0, 20))

    fig.canvas.draw_idle()

roll_slider.on_changed(update)
pitch_slider.on_changed(update)
yaw_slider.on_changed(update)

ax.set_xlim((-30, 30))
ax.set_ylim((-20, 20))
ax.set_zlim((0, 20))

plt.show()

In [ ]:
# Setup plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')  # Full figure for 3D plot

robot = QuadModel()
robot.update_body_pose(0, 0, 0)  # Initial pose
robot.plot_transform_list(ax)

# Animation parameters
frames = 60  # Number of frames
t = np.linspace(0, 2 * np.pi, frames)  # Time parameter for one cycle

def update(frame):
    ax.cla()  # Clear previous frame

    # Define oscillating motion
    roll = 0.05 * np.pi * np.sin(t[frame]) 
    pitch = 0.04 * np.pi * np.sin(t[frame] + np.pi/2)
    yaw = 0.03 * np.pi * np.cos(t[frame])
    robot.update_body_pose(roll, pitch, yaw)
    robot.plot_transform_list(ax)

# Create animation
ani = FuncAnimation(fig, update, frames=frames, interval=20, repeat=True)
ax.set_xlim((-30, 30))
ax.set_ylim((-20, 20))
ax.set_zlim((0, 20))
HTML(ani.to_jshtml())
# ani.save('quad_animation.mp4')